In [1]:
import pandas as pd
path = '/Users/sonjiyeong/Documents/랩실 프로젝트/스마트폰 리뷰마이닝/2. 속성 기반 감성 분석/output/'

In [2]:
df_p = pd.read_csv(path+'최종_period_데이터프레임.csv')[['id', 'period']]

In [3]:
data = pd.read_csv(path + '감성분석결과_atepc_병합')
df = data.copy()

In [4]:
# 두개 모델 갑성 전처리
df['f_sentiment'] = df.apply(lambda row: row['sentiment'] if pd.isna(row['bf_sentiment']) else row['bf_sentiment'], axis=1)
print('제거 전 데이터 개수:', df.shape[0])
df = df[~((df['feature'] == 'charg') & df['bf_aspect'].isna())] # atepc charge 뽑힌거 아닌거는 다 제외하기 
df = df[df['main_feature']!='other'] # other(분류한 속성이 아닌 것) 제거하기 
print('제거 후 데이터 개수:', df.shape[0])

제거 전 데이터 개수: 103046
제거 후 데이터 개수: 96081


In [5]:
df_sel = df[['id', 'model', 'ori_feature', 'f_sentiment']]
df_sel.head()

,id,model,ori_feature,f_sentiment
0,1,galaxy_s3,quality materials,Negative
1,2,galaxy_s3,memory,Neutral
2,2,galaxy_s3,battery,Positive
3,2,galaxy_s3,battery life,Positive
4,3,galaxy_s3,16gb,Negative


In [6]:
df_asp = pd.read_csv(path + 'apc_훈련데이터셋.csv')
df = pd.merge(df_asp, df_sel, on=['id', 'ori_feature'], how='right')

In [7]:
# 리뷰 등장 문장 전처리
df['mark_sentence'] = df['mark_sentence'].replace('?', '.').replace('!', '.')
df['mark_sentence'] = df['mark_sentence'].apply(lambda x: x.split('.'))

In [8]:
def sentence_select(mark_sentence):
    for sen in mark_sentence:
        if '[B-ASP]' in sen:
            new_sen = sen.replace('[B-ASP]', '').replace('[E-ASP]', '')
            return new_sen

In [9]:
df['sentence'] = df['mark_sentence'].apply(sentence_select)
df = df[['id', 'model', 'sentence', 'ori_feature', 'feature', 'main_feature', 'f_sentiment']]

In [10]:
data = df.copy()
data.head()

,id,model,sentence,ori_feature,feature,main_feature,f_sentiment
0,1,galaxy_s3,not at all sure why samsung ' s best mobile t...,quality materials,qualiti_materi,body,Negative
1,2,galaxy_s3,i like the fact to expand memory by micro sd ...,memory,memori,stor,Neutral
2,2,galaxy_s3,i like the fact to expand memory by micro sd ...,battery,batteri,bat,Positive
3,2,galaxy_s3,battery life is great for me so far looking f...,battery life,batteri_life,bat,Positive
4,3,galaxy_s3,however the expensive galaxy s3 only has 16gb...,16gb,16gb,stor,Negative


In [11]:
data = data[data['main_feature']=='dis']
data = pd.merge(data, df_p[df_p['period']<4], on='id')
data

,id,model,sentence,ori_feature,feature,main_feature,f_sentiment,period
0,5,galaxy_s3,camera speed size screen feel look all super,size screen,size_screen,dis,Positive,1
1,6,galaxy_s3,i really like the s2 screen,screen,screen,dis,Positive,1
2,6,galaxy_s3,i do like the fact the screen is bigger on th...,the screen,screen,dis,Negative,1
3,7,galaxy_s3,first of all to justify the price difference ...,screen,screen,dis,Positive,1
4,7,galaxy_s3,first of all to justify the price difference ...,gorilla glass,gorilla_glass,dis,Positive,1
...,...,...,...,...,...,...,...,...
16448,76145,galaxy_s10e,had scrapes on sides and crack on screen,screen,screen,dis,Negative,3
16449,76145,galaxy_s10e,screen falls off easily and water bubbles for...,screen,screen,dis,Negative,3
16450,76203,galaxy_s10e,"regarding the appearance of the cell phone , ...",the screen,screen,dis,Positive,3
16451,76227,galaxy_s10e,"i loved my s10 + , but the size and curved gl...",glass screen,glass_screen,dis,Positive,3


In [12]:
data.to_csv('tf-idf용_데이터프레임.csv', index=False)